## Import All Required Libraries

In [26]:
# Import all connection libraries

from spotipy import SpotifyClientCredentials
import spotipy
import sqlalchemy as sql

# Import all data manipulation libraries

import pandas as pd
from pandarallel import pandarallel
from tqdm import tqdm
pandarallel.initialize(verbose=0, nb_workers=8)

In [27]:
from functools import lru_cache

# Use the lru_cache decorator to cache the result of the function
# maxsize=1000 means the cache will store the result of up to 1000 items
@lru_cache(maxsize=1000)
def get_playlist_tracks(playlist_uri):
    """
    This function uses the Spotify API to get the tracks of a given playlist.
    The function is decorated with the lru_cache decorator to cache the result for each unique playlist_uri.
    So, if the same playlist_uri is passed to the function again, the cached result will be returned
    instead of making a new API call.
    :param playlist_uri: The Spotify URI of the playlist
    :return: A DataFrame containing the tracks of the playlist
    """
    # Use the Spotify API to get the tracks of the playlist
    tracks = pd.json_normalize(sp.playlist_tracks(playlist_uri), record_path=['items'])
    return tracks

## Establish External Connections

In [28]:
# establish connection to Spotify API

cid = 'e5448a8a4fdc4b5d98b44e956d50546d'
secret = '8924c0394d3f49a4a569fc03e891aa1b'
client_credentials = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials, requests_timeout=15, retries=10)

# establish connection to Postgres

engine = sql.create_engine('postgresql+psycopg2://postgres:DataNerd2023!!\
@localhost/spotify')

## Load Initial Training Data

In [29]:
# load training data
new_batch = pd.read_csv('C:\\Users\\Chase\\OneDrive\\Documents\\UVU-2022-2023\\distinct_playlists_new.csv')[['playlist_uri', 'playlist_name']]
new_batch['playlist_uri'] = new_batch['playlist_uri'].str.strip()

db_query = pd.read_sql('SELECT DISTINCT playlist_uri FROM playlist_tracks', engine)

outer = new_batch.merge(db_query, how='outer', indicator=True)
anti_join = outer[(outer._merge=='left_only')].drop('_merge', axis=1)

new_batch = pd.DataFrame(anti_join)
new_batch = new_batch[0:25]
new_batch.head(20)

,playlist_uri,playlist_name
17574,spotify:playlist:37i9dQZF1EIfFQvdFSIRRl,Clean Workout Upbeat Mix
17577,spotify:playlist:37i9dQZF1EIfglLEyw51KY,Country Hits Mix
17596,spotify:playlist:37i9dQZF1EIfocGsfb5B1R,Clean R&B Mix
17633,spotify:playlist:37i9dQZF1EIfXSWAkNhQbi,2010s EDM Mix
17634,spotify:playlist:37i9dQZF1EIfYEGlxfG5a0,Hip Hop House Mix
17635,spotify:playlist:37i9dQZF1EIfyHGfvPQvQQ,Beach House Mix
17636,spotify:playlist:37i9dQZF1EIfYOWZLm9iq6,Hype Hip Hop Mix
17637,spotify:playlist:37i9dQZF1EIfyXR9EchWmA,German Pop Mix
17638,spotify:playlist:37i9dQZF1EIfzDyEgytsCm,Viral Pop Mix
17639,spotify:playlist:37i9dQZF1EIg0AuSxAydwd,80s Pop Mix


## Iterate Through Training Data to Change Grain of Data

In [30]:
def load_all_data():
        # load training data
        new_batch = pd.read_csv('C:\\Users\\Chase\\OneDrive\\Documents\\UVU-2022-2023\\distinct_playlists_new.csv')[['playlist_uri', 'playlist_name']]
        new_batch['playlist_uri'] = new_batch['playlist_uri'].str.strip()

        db_query = pd.read_sql('SELECT DISTINCT playlist_uri FROM playlist_tracks', engine)

        outer = new_batch.merge(db_query, how='outer', indicator=True)
        anti_join = outer[(outer._merge=='left_only')].drop('_merge', axis=1)

        new_batch = pd.DataFrame(anti_join)
        new_batch = new_batch[0:35]
        
        # extract all tracks in playlists
        load_batch = []
        series = new_batch['playlist_uri'].to_dict()
        for playlist in tqdm(series.values()):
                try:
                        tracks = get_playlist_tracks(playlist)
                        if len(tracks) >= 100:
                                tracks2 = pd.json_normalize(sp.playlist_tracks(playlist, offset=100), record_path=['items'])
                                tracks3 = pd.json_normalize(sp.playlist_tracks(playlist, offset=200), record_path=['items'])
                                tracks = pd.concat([tracks, tracks2, tracks3])
                                secondary_data = pd.json_normalize(sp.playlist(playlist))
                                tracks['playlist_uri'] = playlist
                                tracks = tracks.merge(secondary_data, left_on='playlist_uri', right_on='uri')
                                tracks = tracks.rename(columns={"name":"playlist_name", "track.name":"track_name", "track.uri":"track_uri", "track.album.name":"album_name", "track.explicit":"isExplicit", "track.album.release_date":"release_date", "track.duration_ms":"duration_ms", "track.album.uri":"album_uri", "added_by.external_urls.spotify": "added_by_external_urls_spotify", 'added_by.href':"added_by_href", "added_by.id":"added_by_id", "added_by.type":"added_by_type", "added_by.uri":"added_by_uri", "track.album.album_type":"track_album_album_type", "track.album.external_urls.spotify":"track_album_external_urls_spotify", "track.album.href":"track_album_href", "track.album.id":"track_album_id", "track.album.release_date_precision":"track_album_release_date_precision", "track.album.total_tracks":"track_album_total_tracks","track.album.type":"track_album_type", "track.disc_number":"track_disc_number", "track.episode":"track_episode", "track.external_ids.isrc":"track_external_ids_isrc", "track.external_urls.spotify":"track_external_ids_spotify", "track.href":"track_href", "track.id":"track_id", "track.is_local":"track_is_local", "track.popularity":"track_popularity", "track.preview_url":"track_preview_url", "track.track":"track_track", "track.track_number":"track_track_number", "track.type":"track_type", "video_thumbnail.url":"video_thumbnail_url", "external_urls.spotify":"external_urls_spotify", "followers.href":"followers_href", "followers.total":"followers_total", "owner.display_name":"owner_display_name", "owner.external_urls.spotify":"owner_external_urls_spotify", "owner.href":"owner_href", "owner.id":"owner_id", "owner.type":"owner_type", "owner.uri":"owner_uri", "tracks.href":"tracks_href", "tracks.limit":"tracks_limit", "tracks.next":"tracks_next", "tracks.offset":"tracks_offset", "tracks.previous":"tracks_previous", "tracks.total":"tracks_total"})
                                tracks = tracks.drop(columns=['track.album.artists', 'track.album.available_markets', 'track.album.images', 'track.artists', 'track.available_markets', 'images', 'tracks.items'])

                                load_batch.append(tracks)
                        else:
                                secondary_data = pd.json_normalize(sp.playlist(playlist))
                                tracks['playlist_uri'] = playlist
                                tracks = tracks.merge(secondary_data, left_on='playlist_uri', right_on='uri')
                                tracks = tracks.rename(columns={"name":"playlist_name", "track.name":"track_name", "track.uri":"track_uri", "track.album.name":"album_name", "track.explicit":"isExplicit", "track.album.release_date":"release_date", "track.duration_ms":"duration_ms", "track.album.uri":"album_uri", "added_by.external_urls.spotify": "added_by_external_urls_spotify", 'added_by.href':"added_by_href", "added_by.id":"added_by_id", "added_by.type":"added_by_type", "added_by.uri":"added_by_uri", "track.album.album_type":"track_album_album_type", "track.album.external_urls.spotify":"track_album_external_urls_spotify", "track.album.href":"track_album_href", "track.album.id":"track_album_id", "track.album.release_date_precision":"track_album_release_date_precision", "track.album.total_tracks":"track_album_total_tracks","track.album.type":"track_album_type", "track.disc_number":"track_disc_number", "track.episode":"track_episode", "track.external_ids.isrc":"track_external_ids_isrc", "track.external_urls.spotify":"track_external_ids_spotify", "track.href":"track_href", "track.id":"track_id", "track.is_local":"track_is_local", "track.popularity":"track_popularity", "track.preview_url":"track_preview_url", "track.track":"track_track", "track.track_number":"track_track_number", "track.type":"track_type", "video_thumbnail.url":"video_thumbnail_url", "external_urls.spotify":"external_urls_spotify", "followers.href":"followers_href", "followers.total":"followers_total", "owner.display_name":"owner_display_name", "owner.external_urls.spotify":"owner_external_urls_spotify", "owner.href":"owner_href", "owner.id":"owner_id", "owner.type":"owner_type", "owner.uri":"owner_uri", "tracks.href":"tracks_href", "tracks.limit":"tracks_limit", "tracks.next":"tracks_next", "tracks.offset":"tracks_offset", "tracks.previous":"tracks_previous", "tracks.total":"tracks_total"})
                                tracks = tracks.drop(columns=['track.album.artists', 'track.album.available_markets', 'track.album.images', 'track.artists', 'track.available_markets', 'images', 'tracks.items'])
                                load_batch.append(tracks)
                except:
                        pass
        load_batch = pd.concat(load_batch)
        if set(['track.album.is_playable','track.album.restrictions.reason', 'track.external_ids.spotify']).issubset(load_batch.columns):
                load_batch = load_batch.drop(columns=['track.album.is_playable', 'track.album.restrictions.reason', 'track.external_ids.spotify', 'track.is_playable'])
        else:
                load_batch
        load_batch.to_sql('playlist_tracks', engine, if_exists='append')
        print(f'playlist_tracks updated #{i}')
        
        # extract all artists
        df = pd.read_sql('''SELECT DISTINCT track_uri FROM artists''', engine)
        outer = load_batch.merge(df, how='outer', indicator=True)
        anti_join = outer[(outer._merge=='left_only')].drop('_merge', axis=1)

        df = pd.Series(anti_join['track_uri'])
        base_list = []
        for track in tqdm(df):
                try:
                        df2 = pd.json_normalize(sp.track(track), record_path=['artists'])
                        df2['track_uri'] = track
                        df2 = df2[['name', 'track_uri']]
                        base_list.append(df2)
                except:
                        pass
        df2 = pd.concat(base_list)
        
        df2['RN'] = df2.groupby("track_uri")["name"].rank(method="first", ascending=True)
        df2 = df2.pivot(index='track_uri', columns=['RN'], values='name').reset_index()
        df2 = df2.rename(columns={1.0:'artist1', 2.0:'artist2', 3.0:'artist3', 4.0:'artist4', 5.0:'artist5'})
        df2 = df2[['track_uri', 'artist1', 'artist2', 'artist3', 'artist4', 'artist5']]
        
        df2.to_sql('artists', engine, if_exists='append')
        print(f'artists updated #{i}')
        
        # extract all audio_features
        df = pd.read_sql('''SELECT DISTINCT track_uri FROM audio_features''', engine)
        outer = load_batch.merge(df, how='outer', indicator=True)
        anti_join = outer[(outer._merge=='left_only')].drop('_merge', axis=1)

        df = pd.Series(anti_join['track_uri'])

        base_list = []
        for track in tqdm(df):
                try:
                        df3 = pd.json_normalize(sp.audio_features(track))
                        df3['track_uri'] = track
                        df3 = df3[['track_uri', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']]
                        base_list.append(df3)
                except:
                        pass
        df3 = pd.concat(base_list)
        
        df3.to_sql('audio_features', con=engine, if_exists='append')
        print(f'audio_features updated #{i}')

In [31]:
i = 1

while i <= 8:
    load_all_data()
    i += 1

100%|██████████| 35/35 [00:21<00:00,  1.64it/s]


playlist_tracks updated #1


100%|██████████| 589/589 [01:05<00:00,  8.97it/s]


artists updated #1


100%|██████████| 591/591 [01:01<00:00,  9.61it/s]


...

100%|██████████| 460/460 [00:43<00:00, 10.58it/s]


audio_features updated #2


100%|██████████| 35/35 [00:23<00:00,  1.48it/s]


playlist_tracks updated #3


 56%|█████▌    | 345/616 [00:36<00:29,  9.32it/s]

In [ ]:
all_data = pd.read_sql('''SELECT DISTINCT playlist_uri, playlist_name, owner_uri, owner_display_name, collaborative, description, followers_total,
af.track_uri, track_name, artist1, artist2, artist3, artist4, artist5, album_uri, album_name, release_date, 
added_by_uri, added_at, track_album_total_tracks, af.duration_ms, "isExplicit", track_popularity, 
track_preview_url, video_thumbnail_url, danceability, energy, loudness, key, mode, acousticness, speechiness, 
instrumentalness, liveness, valence, tempo, time_signature
FROM playlist_tracks pt
JOIN artists a ON pt.track_uri = a.track_uri
JOIN audio_features af ON pt.track_uri = af.track_uri;''', engine)
f = open("s3.csv", "w")
f.truncate()
f.close()

all_data.to_csv('C:\\Users\\Chase\\OneDrive\\Documents\\Career Development\\Data Science Club\\Spring 2023 Club Project\\s3.csv', mode="w+", index=False)